# Upload Model to S3 Object Storage

This notebook uploads the exported ONNX model to S3-compatible object storage using credentials from the RHOAI data connection.

The model will be uploaded to the path expected by the OpenVINO Model Server deployment configuration.

## 1. Import Required Libraries

Import boto3 for S3 operations and os for reading environment variables.

In [ ]:
import boto3
import os
from pathlib import Path

print(f"boto3 version: {boto3.__version__}")

## 2. Configure S3 Connection

Read S3 credentials and endpoint from environment variables set by the RHOAI data connection.

When you create a data connection in RHOAI, the credentials are automatically injected as environment variables:
- `AWS_ACCESS_KEY_ID`: S3 access key
- `AWS_SECRET_ACCESS_KEY`: S3 secret key
- `AWS_S3_ENDPOINT`: S3 endpoint URL
- `AWS_S3_BUCKET`: S3 bucket name
- `AWS_DEFAULT_REGION`: AWS region (optional)

In [ ]:
# Read S3 configuration from environment variables
aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
aws_s3_endpoint = os.environ.get('AWS_S3_ENDPOINT')
aws_s3_bucket = os.environ.get('AWS_S3_BUCKET')
aws_region = os.environ.get('AWS_DEFAULT_REGION', 'us-east-1')

# Verify required credentials are available
if not all([aws_access_key_id, aws_secret_access_key, aws_s3_endpoint, aws_s3_bucket]):
    raise ValueError("Missing required S3 credentials. Ensure data connection is properly configured.")

print("✓ S3 credentials loaded successfully")
print(f"  Endpoint: {aws_s3_endpoint}")
print(f"  Bucket: {aws_s3_bucket}")
print(f"  Region: {aws_region}")

## 3. Initialize S3 Client

Create a boto3 S3 client configured with the credentials from the data connection.

In [ ]:
# Initialize S3 client
s3_client = boto3.client(
    's3',
    endpoint_url=aws_s3_endpoint,
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_region,
    verify=False  # Disable SSL verification for self-signed certificates in lab environments
)

print("✓ S3 client initialized")

## 4. Upload Model to S3

Upload the ONNX model file to the S3 bucket at the path `models/image-classifier/model.onnx`.

This path matches the configuration expected by the OpenVINO Model Server deployment.

In [ ]:
# Define local file path and S3 destination
local_model_path = Path("models/openvino/model.onnx")
s3_key = "models/image-classifier/model.onnx"

# Verify local file exists
if not local_model_path.exists():
    raise FileNotFoundError(f"Model file not found: {local_model_path}")

print(f"Uploading {local_model_path} to s3://{aws_s3_bucket}/{s3_key}...")

# Upload file to S3
s3_client.upload_file(
    Filename=str(local_model_path),
    Bucket=aws_s3_bucket,
    Key=s3_key
)

print(f"Model uploaded successfully to: {s3_key}")

## 5. Verify Upload

Confirm the model file is accessible in S3 by checking its metadata.

In [ ]:
# Get object metadata to verify upload
try:
    response = s3_client.head_object(Bucket=aws_s3_bucket, Key=s3_key)
    file_size = response['ContentLength']
    file_size_mb = file_size / (1024 * 1024)
    last_modified = response['LastModified']
    
    print("✓ Upload verified successfully")
    print(f"  S3 Path: s3://{aws_s3_bucket}/{s3_key}")
    print(f"  Size: {file_size_mb:.1f} MB")
    print(f"  Last Modified: {last_modified}")
except Exception as e:
    print(f"✗ Error verifying upload: {e}")

## Summary

You have successfully:
1. Configured S3 client using credentials from the RHOAI data connection
2. Uploaded the ONNX model to `models/image-classifier/model.onnx`
3. Verified the upload was successful

The model is now ready to be deployed using OpenVINO Model Server in the next steps of the exercise.